# **INF8460 A20 Project: Open-domain questions answering**

<br>

Equipe 8:


*   Cedric Sadeu (Glove, ranking with classification)
*   Mamoudou Sacko (pretraitement + TF-IDF, cosine ranking)
*   Oumayma Messoussi (PCP Bert, ML/DL for ranking)

<br>

---

<br>

In [1]:
!pip install sent2vec transformers pytorch-pretrained-bert # pytorch-nlp pytorch_transformers

     |████████████████████████████████| 1.3MB 7.9MB/s 
     |████████████████████████████████| 133kB 38.0MB/s 
     |████████████████████████████████| 2.9MB 40.9MB/s 
     |████████████████████████████████| 1.1MB 37.6MB/s 
     |████████████████████████████████| 890kB 46.1MB/s 
     |████████████████████████████████| 133kB 59.6MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 6.9MB 32.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9521c72f50ebdf6f51b1403e4ba5962bf1ebb39b0b5e954107cdc17aefb7d858
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.25 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import io
import os
import math
import nltk
import time
import torch
import random
import sklearn
import zipfile
import operator
import requests
import functools
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
import multiprocessing
from functools import partial
from typing import Dict, List, Tuple
from collections import Counter, defaultdict
from concurrent.futures import ProcessPoolExecutor
from scipy.spatial.distance import euclidean, cosine
from transformers import pipeline, Trainer, TrainingArguments
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, BertTokenizer, BertModel, BertForQuestionAnswering

nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/Colab Notebooks/INF8460/Project/'

Mounted at /content/drive
 correction_script.py		   'mlp_regression_msmarco_tfidf_4957*2.h5'
 create_embeddings.py		    MSMARCO
 data				    output
 data_handling.py		    __pycache__
 inf8460_projet_A20_equipe8.ipynb   pytorch_model.bin
 LSTMSiameseTextSimilarity


In [4]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/')

### Lecture des donnees

In [5]:
from data_handling import *

data_path = "data"
output_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output"

train_data = read_data(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/corpus.csv"))
# train_ids = read_questions(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/train_ids.csv"))
val_ids = read_questions(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/val_ids.csv"))
# test_ids = read_questions(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/test.csv"))


paragraphs = [" ".join(sentence.split()).lower() for sentence in train_data[1]]
# questions = [" ".join(sentence.split()).lower() for sentence in train_ids[1]]
val_questions = [" ".join(sentence.split()).lower() for sentence in val_ids[1]]
# test_questions = [" ".join(sentence.split()).lower() for sentence in test_ids[1]]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Pretraitement

In [6]:
from data_handling import *
pre = Preprocess()

paragraphs_tokenized = pre.preprocess_pipeline(paragraphs)
# questions_tokenized = pre.preprocess_pipeline(questions)
val_questions_tokenized = pre.preprocess_pipeline(val_questions)
# test_questions_tokenized = pre.preprocess_pipeline(test_questions)

paragraphs_text = [" ".join(sentence) for sentence in paragraphs_tokenized]
# questions_text = [" ".join(sentence) for sentence in questions_tokenized]
val_questions_text = [" ".join(sentence) for sentence in val_questions_tokenized]
# test_questions_text = [" ".join(sentence) for sentence in test_questions_tokenized]

del paragraphs_tokenized
# del questions_tokenized
del val_questions_tokenized
# del test_questions_tokenized



---

<br>

## **1. Plongements lexicaux**

### TF-IDF

In [7]:
from create_embeddings import buildVocab, getTfIdfReprentation, get_doc_embedded, sklearn_svd
from sklearn.feature_extraction.text import TfidfVectorizer

# paragraphs_vocab = buildVocab(paragraphs_text)
# max_feat = int(0.01 * len(paragraphs_vocab))
vectorizer = TfidfVectorizer(max_features=5000) # vocabulary=paragraphs_vocab
paragraphs_tfidf = getTfIdfReprentation(paragraphs_text, vectorizer)

val_questions_tfidf = vectorizer.transform(val_questions_text).todense()

# corpus = {'id': train_data[0], 'paragraph': paragraphs_tfidf}
# save_to_csv("corpus.csv", corpus)

# train_ids = {'id': train_ids[0], 'question': paragraphs_tfidf, 'paragraph_id': train_ids[2], 'answer': train_ids[3] }
# save_to_csv("train_ids.csv", train_ids)

### GloVe

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm glove.6B.50d.txt
!rm glove.6B.100d.txt
!rm glove.6B.200d.txt

In [1]:
from data_handling import read_from_csv
from sklearn.feature_extraction.text import CountVectorizer
from create_embeddings import get_gloves_dict, get_lines_gloves, get_plong_corpus

path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/corpus.csv"
datat = read_from_csv(path)

vectorizer = CountVectorizer()
X = vectorizer.fit(datat[1]).vocabulary_

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
glove_dict = get_gloves_dict()
key_set = set(X.keys()) & set(glove_dict.keys())
glove_dict_vocab_corpus = {key: glove_dict[key] for key in key_set}

In [3]:
plongement_doc = get_plong_corpus(datat[1], glove_dict_vocab_corpus)

### Plongements contextuels pré-entraînés

> #### BERT base pre-trained

In [ ]:
torch.cuda.set_device(0)

questions = ["How many parameters does BERT-large have?"]
answer_text = r"""BERT-large is really big... it has 24-layers and an embedding size of 1,024, 
                  for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take 
                  a couple minutes to download to your Colab instance."""

model = BertModel.from_pretrained("bert-base-cased", return_dict=True)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
# # model = model.to(device)

for question in questions:
    inputs = tokenizer(question, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # display tokens and ids
    for token, id in zip(text_tokens, input_ids):
        if id == tokenizer.sep_token_id:
            print('')
        print('{:<12} {:>6,}'.format(token, id))
        if id == tokenizer.sep_token_id:
            print('')

    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

> #### spacy BERT

In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

sentences = [
    "This is an awesome book to learn NLP.",
    "DistilBERT is an amazing NLP model.",
    "We can interchangeably use embedding, encoding, or vectorizing.",
]

vectorizer = Vectorizer()
vectorizer.bert(sentences)
vectors_bert = vectorizer.vectors



---

<br>

## **2. Ordonnancement**



> #### cosine similarity



In [ ]:
def voisins(word, df, n, distfunc=cosine):
    assert distfunc.__name__ == 'cosine' or distfunc.__name__ == 'euclidean', "distance metric not supported"
    order = True if distfunc.__name__ == 'euclidean' else False

    closest = {}
    for w in df:
        distance = distfunc(word, df[w])
        closest[w] = distance

    closest = {k: v for k, v in sorted(closest.items(), key=lambda item: item[1], reverse=order)}

    return list(closest.keys())[:n], list(closest.values())[:n]

In [ ]:
# paragraphs_vocab = buildVocab(paragraphs_text)
questions_vocab = buildVocab(questions_text)

questions_ids = train_ids[0]
vectorizer = TfidfVectorizer(max_features=500) # vocabulary=questions_vocab
questions_tfidf = getTfIdfReprentation(questions_text, vectorizer)

dic_questions = {}
for i, ids in enumerate(questions) :
    dic_questions[questions_ids[i]] = questions_tfidf[i]

#questions_tfidf_r = sklearn_svd(questions_tfidf, len(embeddings))

In [ ]:
start = time.time()

ranking_list = {}
for i in range(len(test_questions_text)):
    new_question = test_questions_text[i]
    new_question_tokenized = pre.preprocess_pipeline([new_question])
    new_question_text = [" ".join(sentence) for sentence in new_question_tokenized]
    
    new_question_tfidf = vectorizer.transform(new_question_text).todense()

    topk_ids, topk_questions = voisins(new_question_tfidf, dic_questions, 3, distfunc=cosine)
    print(topk_ids, topk_questions)

    ranking_list[i] = topk_ids

print(time.time() - start)

In [ ]:
# save to file
save_to_csv("test_cosine_sim_questions.csv", ranking_list)



> #### LSTM Siamese text similarity



In [ ]:
# !git clone https://github.com/amansrivastava17/lstm-siamese-text-similarity.git

In [ ]:
!wget https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/LSTMSiameseTextSimilarity/')

In [ ]:
from model import SiameseBiLSTM
from inputHandler import word_embed_meta_data, create_test_data
from config import siamese_config
import pandas as pd

############ Data Preperation ##########

# df = pd.read_csv('lstm-siamese-text-similarity/sample_data.csv')
df = pd.read_csv('SICK_train.txt', names=['pair_ID', 'sentence_A', 'sentence_B', 'relatedness_score', 'entailment_judgment'], 
                 skiprows=1, sep='\t')
df.drop('pair_ID', axis=1, inplace=True)
df.drop(df[df.entailment_judgment == 'CONTRADICTION'].index, inplace=True)
print(df.head())

sentences1 = list(df['sentence_A'])
sentences2 = list(df['sentence_B'])
is_similar = list(df['relatedness_score'])
del df

                                          sentence_A  ... entailment_judgment
0  A group of kids is playing in a yard and an ol...  ...             NEUTRAL
1  A group of children is playing in the house an...  ...             NEUTRAL
2  The young boys are playing outdoors and the ma...  ...          ENTAILMENT
3  The kids are playing outdoors near a man with ...  ...             NEUTRAL
4  The young boys are playing outdoors and the ma...  ...             NEUTRAL

[5 rows x 4 columns]


In [ ]:
######## Word Embedding ############

tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'], glove_dict)

embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
del sentences1
del sentences2

######## Training ########

class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()

CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)

best_model_path = siamese.train_model(sentences_pair, is_similar, embedding_meta_data, model_save_directory='./')

Embedding matrix shape: (2277, 300)
vector not found for word - dog,
vector not found for word - isn't
vector not found for word - man's
vector not found for word - black,
vector not found for word - corndogs
vector not found for word - little,
vector not found for word - young,
vector not found for word - white,
vector not found for word - small,
vector not found for word - girl,
vector not found for word - red,
vector not found for word - woman,
vector not found for word - uniform,
vector not found for word - child,
vector not found for word - rope,
vector not found for word - wagon,
vector not found for word - field,
vector not found for word - boy,
vector not found for word - backbends
vector not found for word - necklace,
vector not found for word - tree,
vector not found for word - pine,
vector not found for word - snow,
vector not found for word - blue,
vector not found for word - gray,
vector not found for word - big,
vector not found for word - dog's
vector not found for word 

In [ ]:
######## Testing Q/Q pairs ########

from operator import itemgetter
from keras.models import load_model

model = load_model(best_model_path)

test_sentence_pairs = [('A man with a jersey is dunking the ball at a basketball game', 'The ball is being dunked by a man with a jersey at a basketball game'),
                       ('Two dogs are fighting', 'Two dogs are wrestling and hugging'), 
                       ('A group of kids is playing in a yard and an old man is standing in the background', 'A group of boys in a yard is playing and a man is standing in the background'), 
                       ('What can make Physics easy to learn?','How can you make physics easy to learn?'),
                       ('How many times a day do a clocks hands overlap?','What does it mean that every time I look at the clock the numbers are the same?')]

test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])

preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
results.sort(key=itemgetter(2), reverse=True)
print(results)

1/1 [==============================] - 0s 7ms/step
[('A man with a jersey is dunking the ball at a basketball game', 'The ball is being dunked by a man with a jersey at a basketball game', 4.673317), ('A group of kids is playing in a yard and an old man is standing in the background', 'A group of boys in a yard is playing and a man is standing in the background', 3.9694657), ('Two dogs are fighting', 'Two dogs are wrestling and hugging', 3.3925214), ('What can make Physics easy to learn?', 'How can you make physics easy to learn?', 3.1973097), ('How many times a day do a clocks hands overlap?', 'What does it mean that every time I look at the clock the numbers are the same?', 1.8688544)]


In [ ]:
for i, doc in enumerate(paragraphs_text):
    test_sentence_pairs = [(doc, passage) for passage in set(paragraphs_text[:i]+paragraphs_text[i+1:])]

    test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer, test_sentence_pairs, siamese_config['MAX_SEQUENCE_LENGTH'])

    preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
    results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
    results.sort(key=itemgetter(2), reverse=True)
    
    print(doc)
    print('\n')
    for line in results[:10]:
        print(line[1], line[2])

    break

2096/2096 [==============================] - 61s 29ms/step
the norman norman nourmands french normands latin normanni were the people who in the and century gave their name to normandy region in france they were descended from norse norman come from norseman raider and pirate from denmark iceland and norway who under their leader rollo agreed to swear fealty to king charles iii of west francia through generation of assimilation and mixing with the native frankish and roman gaulish population their descendant would gradually merge with the carolingian based culture of west francia the distinct cultural and ethnic identity of the norman emerged initially in the first half of the century and it continued to evolve over the succeeding century


both the ford foundation and the rockefeller foundation were heavily involved one key leader wa norman borlaug the father of the green revolution who received the nobel peace prize in he is credited with saving over billion people from starvation th



> #### MLP regression model



In [8]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/MSMARCO/msmarco-docdev-100k.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.iloc[:25000,:] 
print(len(df))

df_score = df['score']
normalized_df = (df_score - df_score.min())/ (df_score.max()-df_score.min()) #df_score.std()
del df_score
print(normalized_df.head())

y = normalized_df#.multiply(1.0/df["rank"], axis="index")
# print(y.head())
del normalized_df

df.drop(['rank', 'score'], axis=1, inplace=True)

# Concatenate queries and docs in pairs

# X = df[['query', 'doc']].agg(' [sep] '.join, axis=1)
# del df

25000
0    0.761969
1    0.759416
2    0.755649
3    0.748219
4    0.746034
Name: score, dtype: float64


In [9]:
from data_handling import *
pre = Preprocess()

paragraphs_tokenized = pre.preprocess_pipeline(df["doc"])
questions_tokenized = pre.preprocess_pipeline(df["query"])

paragraphs_text = [" ".join(sentence) for sentence in paragraphs_tokenized]
questions_text = [" ".join(sentence) for sentence in questions_tokenized]

del paragraphs_tokenized
del questions_tokenized
del df

In [10]:
# tfidf embeddings

from create_embeddings import buildVocab, getTfIdfReprentation, get_doc_embedded, sklearn_svd
from sklearn.feature_extraction.text import TfidfVectorizer

# paragraphs_vocab = buildVocab(paragraphs_text) # taille du vocab: 495737
# max_feat = int(0.01 * len(paragraphs_vocab))
# vectorizer = TfidfVectorizer(max_features=max_feat) # vocabulary=paragraphs_vocab
msmarco_paragraphs_tfidf = getTfIdfReprentation(paragraphs_text, vectorizer) # meme vectorizer utilisé pour notre corpus

# new_question = vals_questions_text[i]
# new_question_tokenized = pre.preprocess_pipeline([new_question])
# new_question_text = [" ".join(sentence) for sentence in new_question_tokenized]

msmarco_questions_tfidf = vectorizer.transform(questions_text).todense()

In [4]:
# glove embeddings

import pickle
from sklearn.feature_extraction.text import CountVectorizer
from create_embeddings import get_gloves_dict, get_lines_gloves, get_plong_corpus

vectorizer = CountVectorizer()
vocab_queries = vectorizer.fit(questions_text).vocabulary_

glove_dict = get_gloves_dict()
key_set = set(vocab_queries.keys()) & set(glove_dict.keys())
glove_dict_vocab_corpus = {key: glove_dict[key] for key in key_set}
del vocab_queries
plongement_queries = get_plong_corpus(questions_text, glove_dict_vocab_corpus)

with open('glove_query_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(plongement_queries, fp)
# del plongement_queries
# df.drop(['query'], axis=1, inplace=True)

/content/create_embeddings.py:123: RuntimeWarning: invalid value encountered in true_divide
  return vec / sum(temp_.values())
/content/create_embeddings.py:123: RuntimeWarning: invalid value encountered in true_divide
  return vec / sum(temp_.values())


In [3]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from create_embeddings import get_gloves_dict, get_lines_gloves, get_plong_corpus

vectorizer = CountVectorizer()
vocab_docs = vectorizer.fit(paragraphs_text).vocabulary_

glove_dict = get_gloves_dict()
key_set = set(vocab_docs.keys()) & set(glove_dict.keys())
glove_dict_vocab_corpus = {key: glove_dict[key] for key in key_set}
del vocab_docs
plongement_doc = get_plong_corpus(paragraphs_text, glove_dict_vocab_corpus)

with open('glove_doc_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(plongement_doc, fp)
# del plongement_doc

In [6]:
import pickle

with open('glove_query_emb_msmarco.txt', 'rb') as fp:
    plongement_queries = pickle.load(fp)

with open('glove_doc_emb_msmarco.txt', 'rb') as fp:
    plongement_doc = pickle.load(fp)

In [ ]:
# bert embeddings

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

vectorizer = Vectorizer()
vectorizer.bert(paragraphs_text)
docs_bert = vectorizer.vectors

vectorizer = Vectorizer()
vectorizer.bert(questions_text)
queries_bert = vectorizer.vectors

Token indices sequence length is longer than the specified maximum sequence length for this model (1617 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import pickle

with open('bert_query_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(queries_bert, fp)

with open('bert_doc_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(docs_bert, fp)

In [ ]:
import tensorflow as tf
from keras.layers import Dense, Embedding, Input
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(np.concatenate((plongement_queries, plongement_doc), axis=1), y, test_size=0.10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(np.concatenate((msmarco_questions_tfidf, msmarco_paragraphs_tfidf), axis=1), y, test_size=0.10, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(np.concatenate((queries_bert, docs_bert), axis=1), y, test_size=0.20, random_state=42)

def MLPregression(X, y, ndims):
    model = Sequential()
    model.add(Dense(100, input_dim=ndims, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mse'])

    # stop = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
    model.fit(X, y, epochs=25, batch_size=100, validation_split=0.15) #, callbacks=[stop])
    return model
    
MLPmodel = MLPregression(X_train, np.array(y_train), max_feat*2)
preds = MLPmodel.predict(X_test)

MLPmodel.save('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/mlp_regression_msmarco_tfidf_8000*2.h5')

In [12]:
print(np.array(y_test)[:5])
print(preds[:5])

[0.74325098 0.78950238 0.78060363 0.78547908 0.74975728]
[[0.7506879 ]
 [0.7999193 ]
 [0.7503954 ]
 [0.78310984]
 [0.7624861 ]]


In [ ]:
import keras

reconstructed_MLPmodel = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/mlp_regression_msmarco_tfidf_4957*2.h5')

for i, query in enumerate(val_questions_tfidf):
    test_sentence_pairs = []
    for doc in paragraphs_tfidf:
        test_sentence_pairs.append(query + doc)
    # np.concatenate([query for i in range(len(paragraphs_tfidf))], paragraphs_tfidf)

    preds = reconstructed_MLPmodel.predict(test_sentence_pairs)

    results = [(i, x, y) for (x, y), z in zip(paragraphs_ids, preds)]
    results.sort(key=itemgetter(2), reverse=True)
    
    print(query)
    print('\n')
    for line in results[:20]:
        print(line[1], line[2])

    break



---

<br>

## **3. Extraction de reponse**



> #### Pytorch BERT squad 2.0



In [ ]:
!git clone https://github.com/surbhardwaj/BERT-QnA-Squad_2.0_Finetuned_Model.git

Cloning into 'BERT-QnA-Squad_2.0_Finetuned_Model'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), done.


In [ ]:
import numpy as np
import pandas as pd
import argparse
from pytorch_pretrained_bert.tokenization import (BasicTokenizer,
                                                  BertTokenizer,whitespace_tokenize)
import collections
import torch
from torch.utils.data import TensorDataset
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering, BertConfig
import math
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from tqdm import tqdm
from termcolor import colored, cprint


class SquadExample(object):
    """
    A single training/test example for the Squad dataset.
    For examples without an answer, the start and end position are -1.
    """

    def __init__(self,
                 example_id,
                 para_text,
                 qas_id,
                 question_text,
                 doc_tokens,
                unique_id):
        self.qas_id = qas_id
        self.question_text = question_text
        self.doc_tokens = doc_tokens
        self.example_id = example_id
        self.para_text = para_text
        self.unique_id = unique_id
        

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "qas_id: %s" % (self.qas_id)
        s += ", question_text: %s" % (
            self.question_text)
        s += ", doc_tokens: [%s]" % (" ".join(self.doc_tokens))
        
        return s



### Convert paragraph to tokens and returns question_text
def read_squad_examples(input_data):
    """Read a SQuAD json file into a list of SquadExample."""
    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False
    i = 0
    examples = []
    for entry in input_data:
        example_id = entry['id']
        paragraph_text = entry['text']
        doc_tokens = []
        prev_is_whitespace = True
        for c in paragraph_text:
            if is_whitespace(c):
                prev_is_whitespace = True
            else:
                if prev_is_whitespace:
                    doc_tokens.append(c)
                else:
                    doc_tokens[-1] += c
                prev_is_whitespace = False
            
        for qa in entry['ques']:
            qas_id = i
            question_text = qa
            
            
            example = SquadExample(example_id = example_id,
                    qas_id=qas_id,
                    para_text = paragraph_text,               
                    question_text=question_text,
                    doc_tokens=doc_tokens,
                    unique_id = i)
            i+=1
            examples.append(example)

    return examples



def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_is_max_context,
                 token_to_orig_map,
                 input_ids,
                 input_mask,
                 segment_ids):
 
        self.doc_span_index = doc_span_index
        self.unique_id = unique_id
        self.example_index = example_index
        self.tokens = tokens
        self.token_is_max_context = token_is_max_context
        self.token_to_orig_map = token_to_orig_map
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        




def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 doc_stride, max_query_length):
    """Loads a data file into a list of `InputBatch`s."""


    features = []
    unique_id = 1
    for (example_index, example) in enumerate(examples):
        query_tokens = tokenizer.tokenize(example.question_text)
        ### Truncate the query if query length > max_query_length..
        if len(query_tokens) > max_query_length:
            query_tokens = query_tokens[0:max_query_length]

        tok_to_orig_index = []
        orig_to_tok_index = []
        all_doc_tokens = []
        for (i, token) in enumerate(example.doc_tokens):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tok_to_orig_index.append(i)
                all_doc_tokens.append(sub_token)

        tok_start_position = None
        tok_end_position = None

        max_tokens_for_doc = max_seq_length - len(query_tokens) - 3
    

        # We can have documents that are longer than the maximum sequence length.
        # To deal with this we do a sliding window approach, where we take chunks
        # of the up to our max length with a stride of `doc_stride`.
        _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
            "DocSpan", ["start", "length"])
        doc_spans = []
        start_offset = 0
        while start_offset < len(all_doc_tokens):
            length = len(all_doc_tokens) - start_offset
            if length > max_tokens_for_doc:
                length = max_tokens_for_doc
            doc_spans.append(_DocSpan(start=start_offset, length=length))
            if start_offset + length == len(all_doc_tokens):
                break
            start_offset += min(length, doc_stride)

        for (doc_span_index, doc_span) in enumerate(doc_spans):
            tokens = []
            token_to_orig_map = {}
            token_is_max_context = {}
            segment_ids = []
            tokens.append("[CLS]")
            segment_ids.append(0)
            for token in query_tokens:
                tokens.append(token)
                segment_ids.append(0)
            tokens.append("[SEP]")
            segment_ids.append(0)

            for i in range(doc_span.length):
                split_token_index = doc_span.start + i
                token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

                is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                       split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)


            input_ids = tokenizer.convert_tokens_to_ids(tokens)

                # The mask has 1 for real tokens and 0 for padding tokens. Only real
                # tokens are attended to.
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            
            features.append(InputFeatures(unique_id = unique_id,
                            example_index = example_index,
                            doc_span_index=doc_span_index,
                            tokens=tokens,   
                            token_is_max_context=token_is_max_context,
                            token_to_orig_map=token_to_orig_map,
                            input_ids=input_ids,
                            input_mask=input_mask,
                            segment_ids=segment_ids))
            unique_id += 1

            
    
    return features


def _get_best_indexes(logits, n_best_size):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)
    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
   
    return best_indexes



def get_final_text(pred_text, orig_text, do_lower_case, verbose_logging=False):
    """Project the tokenized prediction back to the original text."""

    def _strip_spaces(text):
        ns_chars = []
        ns_to_s_map = collections.OrderedDict()
        for (i, c) in enumerate(text):
            if c == " ":
                continue
            ns_to_s_map[len(ns_chars)] = i
            ns_chars.append(c)
        ns_text = "".join(ns_chars)
        return (ns_text, ns_to_s_map)

    
    tokenizer = BasicTokenizer(do_lower_case=do_lower_case)

    tok_text = " ".join(tokenizer.tokenize(orig_text))

    start_position = tok_text.find(pred_text)
    if start_position == -1:
        if verbose_logging:
            logger.info(
                "Unable to find text: '%s' in '%s'" % (pred_text, orig_text))
            print("no answer")
        return orig_text
    end_position = start_position + len(pred_text) - 1

    (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)
    (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)

    if len(orig_ns_text) != len(tok_ns_text):
        if verbose_logging:
            logger.info("Length not equal after stripping spaces: '%s' vs '%s'",
                        orig_ns_text, tok_ns_text)
        return orig_text

    # We then project the characters in `pred_text` back to `orig_text` using
    # the character-to-character alignment.
    tok_s_to_ns_map = {}
    for (i, tok_index) in tok_ns_to_s_map.items():
        tok_s_to_ns_map[tok_index] = i

    orig_start_position = None
    if start_position in tok_s_to_ns_map:
        ns_start_position = tok_s_to_ns_map[start_position]
        if ns_start_position in orig_ns_to_s_map:
            orig_start_position = orig_ns_to_s_map[ns_start_position]

    if orig_start_position is None:
        if verbose_logging:
            logger.info("Couldn't map start position")
        return orig_text

    orig_end_position = None
    if end_position in tok_s_to_ns_map:
        ns_end_position = tok_s_to_ns_map[end_position]
        if ns_end_position in orig_ns_to_s_map:
            orig_end_position = orig_ns_to_s_map[ns_end_position]

    if orig_end_position is None:
        if verbose_logging:
            logger.info("Couldn't map end position")
        return orig_text

    output_text = orig_text[orig_start_position:(orig_end_position + 1)]
    return output_text

    
    
_PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "PrelimPrediction",
["feature_index", "start_index", "end_index", "start_logit", "end_logit"])


def _compute_softmax(scores):
    """Compute softmax probability over raw logits."""
    if not scores:
        return []

    max_score = None
    for score in scores:
        if max_score is None or score > max_score:
            max_score = score

    exp_scores = []
    total_sum = 0.0
    for score in scores:
        x = math.exp(score - max_score)
        exp_scores.append(x)
        total_sum += x

    probs = []
    for score in exp_scores:
        probs.append(score / total_sum)
    return probs


_NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "NbestPrediction", ["text", "start_logit", "end_logit"])

def predict(examples, all_features, all_results, max_answer_length, thresh):

    n_best_size = 10
    
    ### Adding index to feature ###
    example_index_to_features = collections.defaultdict(list)
    for feature in all_features:
        example_index_to_features[feature.example_index].append(feature)
     
    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result
        
        
    all_predictions = collections.OrderedDict()
   
    
    
    for example in examples:
        index = 0
        features = example_index_to_features[example.unique_id]
        prelim_predictions = []
       
        for (feature_index, feature) in enumerate(features):
            result = unique_id_to_result[feature.unique_id]
            start_indexes = _get_best_indexes(result.start_logits, n_best_size)
            end_indexes = _get_best_indexes(result.end_logits, n_best_size)
            for start_index in start_indexes:
                    for end_index in end_indexes:
                     #### we remove the indexes which are invalid @
                        if start_index >= len(feature.tokens):
                            continue
                        if end_index >= len(feature.tokens):

                            continue
                        if start_index not in feature.token_to_orig_map:
                            continue
                        if end_index not in feature.token_to_orig_map:
                            continue
                        if not feature.token_is_max_context.get(start_index, False):
                            continue
                        if end_index < start_index:
                            continue
                        length = end_index - start_index + 1
                        if length > max_answer_length:
                            continue

                        prelim_predictions.append(
                                        _PrelimPrediction(
                                            feature_index=feature_index,
                                            start_index=start_index,
                                            end_index=end_index,
                                            start_logit=result.start_logits[start_index],
                                            end_logit=result.end_logits[end_index]))


        prelim_predictions = sorted(
            prelim_predictions,
            key=lambda x: (x.start_logit + x.end_logit),
            reverse=True) 
            
    
         
        seen_predictions = {}
        nbest = []
        for pred in prelim_predictions:
            if len(nbest) >= n_best_size:
                break
                
            feature = features[pred.feature_index]
            if pred.start_index > 0:  # this is a non-null prediction
                tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
                orig_doc_start = feature.token_to_orig_map[pred.start_index]
                orig_doc_end = feature.token_to_orig_map[pred.end_index]
                orig_tokens = example.doc_tokens[orig_doc_start:(orig_doc_end + 1)]
                tok_text = " ".join(tok_tokens)

                # De-tokenize WordPieces that have been split off.
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # Clean whitespace
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())
                orig_text = " ".join(orig_tokens)

                final_text = get_final_text(tok_text, orig_text, True)
                if final_text in seen_predictions:
                    continue

                seen_predictions[final_text] = True
            else:
                final_text = ""
                seen_predictions[final_text] = True

            nbest.append(
                _NbestPrediction(
                    text=final_text,
                    start_logit=pred.start_logit,
                    end_logit=pred.end_logit))
        
        
    
        if not nbest:
                nbest.append(
                    _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

        assert len(nbest) >= 1
        
        
        total_scores = []
        best_non_null_entry = None
        for entry in nbest:
            total_scores.append(entry.start_logit + entry.end_logit)
            if not best_non_null_entry:
                if entry.text:
                    best_non_null_entry = entry

    
        probs = _compute_softmax(total_scores)
        nbest_json = []
        for (i, entry) in enumerate(nbest):

            output = collections.OrderedDict()
            output["text"] = entry.text if probs[i] > thresh else "<No Answer>"
            output["probability"] = probs[i]
            output["start_logit"] = entry.start_logit if probs[i] > thresh else -1
            output["end_logit"] = entry.end_logit if probs[i] > thresh else -1
            nbest_json.append(output)

        assert len(nbest_json) >= 1
        all_predictions[example] = nbest_json[0]["text"]
        index=+1
    return all_predictions
        

                

RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])




def main(para_file="BERT-QnA-Squad_2.0_Finetuned_Model/Input_file.txt"):
    
    # args = parser.parse_args()
    model_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/pytorch_model.bin"
    
    if torch.cuda.is_available:
        print('GPU available')
    else:
        print('Please set GPU')
    device = torch.device("cuda" if torch.cuda.is_available else "cpu")
    n_gpu = torch.cuda.device_count()
    
    ### Raeding paragraph
    f = open(para_file, 'r')
    para = f.read()
    f.close()
    
    ## Reading question
#     f = open(ques_file, 'r')
#     ques = f.read()
#     f.close()
    
    para_list = para.split('\n\n')
    
    input_data = []
    i = 1
    for para in para_list :
        paragraphs = {}
        splits = para.split('\nQuestions:')
        paragraphs['id'] = i
        paragraphs['text'] = splits[0].replace('Paragraph:', '').strip('\n')
        paragraphs['ques']=splits[1].lstrip('\n').split('\n')
        input_data.append(paragraphs)
        i+=1
       
    
    ## input_data is a list of dictionary which has a paragraph and questions

    
    examples = read_squad_examples(input_data)
    # print(examples)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    
    eval_features = convert_examples_to_features(
            examples = examples,
            tokenizer=tokenizer,
            max_seq_length=384,
            doc_stride=128,
            max_query_length=64)
    
    # print(eval_features)
    
    
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
    
    ### Loading Pretrained model for QnA 
    config = BertConfig("BERT-QnA-Squad_2.0_Finetuned_Model//Results/bert_config.json")
    model = BertForQuestionAnswering(config)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.to(device)
   

    pred_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
    # Run prediction for full data
    pred_sampler = SequentialSampler(pred_data)
    pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=9)
    
    predictions = []
    for input_ids, input_mask, segment_ids, example_indices in tqdm(pred_dataloader):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        
        with torch.no_grad():
            batch_start_logits, batch_end_logits = model(input_ids, segment_ids, input_mask)
            
    
        features=[]
        example = []
        all_results = []
       
        for i, example_index in enumerate(example_indices):
                start_logits = batch_start_logits[i].detach().cpu().tolist()
                end_logits =   batch_end_logits[i].detach().cpu().tolist()
                feature = eval_features[example_index.item()]
                unique_id = int(feature.unique_id)
                features.append(feature)
                all_results.append(RawResult(unique_id=unique_id,
                                             start_logits=start_logits,
                                             end_logits=end_logits))
                
       
        output = predict(examples, features, all_results, 30, 0.25)
        print('Output:\n')
        print(output)
        print('\n')
        predictions.append(output)
 
   
    ### For printing the results ####
    index = None
    for example in examples:
        if index!= example.example_id:
            # print(example.para_text)
            index = example.example_id
            print('\n')
            print(colored('***********Question and Answers *************', 'red'))
          
        ques_text = colored(example.question_text, 'blue')
        print(ques_text)
        prediction = colored(predictions[math.floor(example.unique_id/12)][example], 'green', attrs=['reverse', 'blink'])
        print(prediction)
        print('\n')

In [ ]:
main()

GPU available


100%|██████████| 1/1 [00:00<00:00,  5.38it/s]

Output:

OrderedDict([(qas_id: 0, question_text: What is the formula of alkyl group?, doc_tokens: [Ethers are a class of organic compounds that contain an ether group—an oxygen atom connected to two alkyl or aryl groups. They have the general formula R–O–R′, where R and R′ represent the alkyl or aryl groups. Ethers can again be classified into two varieties: if the alkyl groups are the same on both sides of the oxygen atom, then it is a simple or symmetrical ether, whereas if they are different, the ethers are called mixed or unsymmetrical ethers. A typical example of the first group is the solvent and anesthetic diethyl ether, commonly referred to simply as "ether" (CH3–CH2–O–CH2–CH3). Ethers are common in organic chemistry and even more prevalent in biochemistry, as they are common linkages in carbohydrates and lignin.], 'R–O–R′'), (qas_id: 1, question_text: What is symmetrical ether ?, doc_tokens: [Ethers are a class of organic compounds that contain an ether group—an oxygen atom co

In [ ]:
main("sample_project.txt")

GPU available


100%|██████████| 1/1 [00:00<00:00,  4.60it/s]

Output:

OrderedDict([(qas_id: 0, question_text: Who heads an office or division?, doc_tokens: [In 1982, Regis McKenna was brought in to shape the marketing and launch of the Macintosh. Later the Regis McKenna team grew to include Jane Anderson, Katie Cadigan and Andy Cunningham, who eventually led the Apple account for the agency. Cunningham and Anderson were the primary authors of the Macintosh launch plan. The launch of the Macintosh pioneered many different tactics that are used today in launching technology products, including the "multiple exclusive," event marketing (credited to John Sculley, who brought the concept over from Pepsi), creating a mystique around a product and giving an inside look into a product's creation.], 'Regis McKenna'), (qas_id: 1, question_text: Who leaders the sub-divisions of offices or divisions?, doc_tokens: [In 1982, Regis McKenna was brought in to shape the marketing and launch of the Macintosh. Later the Regis McKenna team grew to include Jane Ander